In [14]:
import urllib2
import re
import re
from bs4 import BeautifulSoup
import json
import datetime
from dateutil.parser import parse
from bs4.element import Comment
import pandas as pd
import collections

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def get_ascii_only(string):
    return ''.join([i if ord(i) < 128 else '' for i in string])
    

def text_from_html(body):
    # soup = BeautifulSoup(body, 'html.parser')
    try:
        texts = body.findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        return u" ".join(t.strip() for t in visible_texts)
    except:
        return ""

def parse_data_save(links):
    d_title = collections.defaultdict(int)
    result = []
    for title, link in links:
        if title in ("If you are interested in editing this book", "If you have questions related to Java", "Recent changes"):
            continue
        page = urllib2.urlopen(link)
        soup = BeautifulSoup(page)
        text = ""
        t = ""
        content = soup.find('div', class_="mw-parser-output")
        for tag in content:
            if tag.name == 'table':
                continue
            if tag.name in ('h1','h2', 'h3', 'h4', 'h5'):
                d = {}
                d["url"] = link
                titt = title + " " + t
                titt = titt.replace("[edit]", "")
                d['title'] = titt
                d['text'] = text
                if text:
                    result.append(d)
                text = ""
                t = tag.text
            else:
                text += get_ascii_only(text_from_html(tag).strip())
    df = pd.DataFrame(result)
    df.to_csv('data.csv')
    return df

def remove_spaces(text):
    return re.sub('\s+',' ',text)

def get_all_wikibooks_links(wikibooks_url):
    page = urllib2.urlopen(wikibooks_url)
    soup = BeautifulSoup(page)
    main_div = soup.find('div', class_="mw-content-ltr")
    ul = main_div.find_all('ul')
    domain = "https://en.wikibooks.org/%s"
    links = []
    for u in ul:
        li = u.find_all('li')
        for l in li:
            a_tag = l.find('a', recursive=False)
            if a_tag: 
                link_text = a_tag.text
                link = domain%a_tag.get('href')
                links.append((link_text, link))
    return links
    

wikibooks_url = 'https://en.wikibooks.org/wiki/Java_Programming'
df = parse_data_save(get_all_wikibooks_links(wikibooks_url))

In [15]:
df.to_csv('data.csv')

In [16]:
df

,text,title,url
0,These conventions are aimed to increase the re...,How to format the content,https://en.wikibooks.org//wiki/Java_Programmin...
1,Most of the time a page from the Java Programm...,How to format the content Page formatting,https://en.wikibooks.org//wiki/Java_Programmin...
2,Sometimes we wish to convey an entire java fil...,How to format the content Java file,https://en.wikibooks.org//wiki/Java_Programmin...
3,"When just a small section of code is shown, th...",How to format the content Snippets,https://en.wikibooks.org//wiki/Java_Programmin...
4,"To represent what is shown on the screen, use ...",How to format the content Screen print,https://en.wikibooks.org//wiki/Java_Programmin...
5,Notes grant readers additional information abo...,How to format the content Notes,https://en.wikibooks.org//wiki/Java_Programmin...
6,To warn the user of things like common pitfall...,How to format the content Warning,https://en.wikibooks.org//wiki/Java_Programmin...
7,If there are sections of the book that require...,How to format the content To-do items,https://en.wikibooks.org//wiki/Java_Programmin...
8,"If a section is optional, it can be hidden lik...",How to format the content Hidden section,https://en.wikibooks.org//wiki/Java_Programmin...
9,This book uses various illustrations and diagr...,How to format the content Illustrations and di...,https://en.wikibooks.org//wiki/Java_Programmin...


In [17]:
json_data = df.to_json(orient='records')

In [18]:
json

<module 'json' from '/usr/lib/python2.7/json/__init__.pyc'>

In [19]:
import json
with open('data.json', 'w') as outfile:
    json.dump(json.loads(json_data), outfile)

In [ ]:
json.loads(json_data)[0]

In [ ]:
df[df['url'] == 'https://en.wikibooks.org//wiki/Java_Programming/Map']

In [ ]:
df.iloc[299].text

In [ ]:
import urllib2
import re
import re
from bs4 import BeautifulSoup
import json
import datetime
from dateutil.parser import parse
from bs4.element import Comment
import pandas as pd
import collections

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def get_ascii_only(string):
    return ''.join([i if ord(i) < 128 else '' for i in string])
    

def text_from_html(body):
    # soup = BeautifulSoup(body, 'html.parser')
    try:
        texts = body.findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        return u" ".join(t.strip() for t in visible_texts)
    except:
        return ""

def parse_data_save(links):
    d_title = collections.defaultdict(int)
    result = []
    for title, link in links:
        if title in ("If you are interested in editing this book", "If you have questions related to Java"):
            continue
        page = urllib2.urlopen(link)
        soup = BeautifulSoup(page)
        text = ""
        t = ""
        for tag in content:
            if tag.name == 'table':
                continue
            if tag.name in ('h1','h2', 'h3', 'h4', 'h5'):
                d = {}
                d["url"] = link
                titt = title + " " + t
                titt = titt.replace("[edit]", "")
                d['title'] = titt
                d['text'] = text
                if text:
                    result.append(d)
                text = ""
                t = tag.text
            else:
                text += get_ascii_only(text_from_html(tag).strip())
    df = pd.DataFrame(result)
    df.to_csv('data.csv')
    return df

def remove_spaces(text):
    return re.sub('\s+',' ',text)

def get_all_wikibooks_links(wikibooks_url):
    page = urllib2.urlopen(wikibooks_url)
    soup = BeautifulSoup(page)
    main_div = soup.find('div', class_="mw-content-ltr")
    ul = main_div.find_all('ul')
    domain = "https://en.wikibooks.org/%s"
    links = []
    for u in ul:
        li = u.find_all('li')
        for l in li:
            a_tag = l.find('a', recursive=False)
            if a_tag: 
                link_text = a_tag.text
                link = domain%a_tag.get('href')
                links.append((link_text, link))
    return links
    

wikibooks_url = 'https://en.wikibooks.org/wiki/Java_Programming'
df = parse_data_save(get_all_wikibooks_links(wikibooks_url))